# Clustering Example Code Along

### 1. Import koniecznych pakietów i wstępna obróbka danych

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

Import danych z pliku CSV:

In [3]:
dataset = spark.read.csv('seeds_dataset.csv', header=True, inferSchema=True)

In [4]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



Algorytm KMeans jest przykładem algorytmu bez nadzoru. Wśród danych brak jest kolumny 'label'. 

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [8]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [9]:
final_data = assembler.transform(dataset)

In [10]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



Przy okazji tego zadania poćwiczymy skalowanie danych.

In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', )

In [13]:
scaler_model = scaler.fit(final_data)

In [14]:
final_data = scaler_model.transform(final_data)

In [15]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

### 2. Tworzenie i trening modelu

Z opisu zadania wiemy, że w eksperymencie zajmowano się trzema rodzajami ziaren. Z tego powodu będziemy dzielić nasz zbiór danych na trzy klastry. 

In [5]:
from pyspark.ml.clustering import KMeans

In [16]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [17]:
model = kmeans.fit(final_data)

In [19]:
print('WSSSE:')
print(model.computeCost(final_data))

WSSSE:
428.60820118716356


In [20]:
centers = model.clusterCenters()

In [21]:
print(centers)

[array([  4.07497225,  10.14410142,  35.89816849,  11.80812742,
         7.54416916,   3.15410901,  10.38031464]), array([  6.35645488,  12.40730852,  37.41990178,  13.93860446,
         9.7892399 ,   2.41585013,  12.29286107]), array([  4.96198582,  10.97871333,  37.30930808,  12.44647267,
         8.62880781,   1.80061978,  10.41913733])]


In [23]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows

